In [1]:
import pyspark  
sc = pyspark.SparkContext('local[*]') 
# do something to prove it works 
rdd = sc.parallelize(range(1000)) 
rdd.takeSample(False, 5) 

[773, 546, 975, 55, 797]

Write a function to format the file in a way that each appropriate value will correspond to only one column. If the data itself contains the splitter comma, it should be quoted by double quotes.  

In [6]:
from pyspark.sql.functions import col, lit, regexp_replace, split, concat, when,expr
from pyspark.sql import SparkSession
import pandas as pd

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Read CSV Example") \
    .getOrCreate()

def get_format_file():
    # Read the text file
    file_read = spark.read.text("clean_me.csv")
    # Format changues
    splited_df_1 = file_read.withColumn("splited_colum",split(file_read["value"],","))
    splited_df_1 = splited_df_1.withColumn("order_id", col("splited_colum")[0])
    splited_df_1 = splited_df_1.withColumn("delivery_company", col("splited_colum")[1])
    splited_df_1 = splited_df_1.withColumn("quantity", when(col("splited_colum")[2].cast("int").isNotNull(),
                                                            col("splited_colum")[2]) \
                                                            .otherwise(lit(0))) 
    splited_df_1 = splited_df_1.withColumn("price",  when(col("splited_colum")[4].cast("int").isNotNull(),
                                                        concat(col("splited_colum")[3],lit("."),col("splited_colum")[4])) 
                                                        .otherwise(col("splited_colum")[3])) 
    splited_df_1 = splited_df_1.withColumn("ordered_date", when(col("splited_colum")[4].cast("int").isNotNull(),
                                                        concat(col("splited_colum")[5])) \
                                                        .otherwise(col("splited_colum")[4]))
    
    splited_df_1 = splited_df_1.withColumn("address", concat(col("splited_colum")[6],lit(","),col("splited_colum")[7]))
    # Delete first record that was as header
    final_df = splited_df_1.select("order_id","delivery_company","quantity","price","ordered_date","address") \
        .filter(col("order_id") != "order_id")
    # Format changues
    return final_df
    
# Call the function to execute the code
get_format_file().show()

+--------+----------------+--------+--------+------------+--------------------+
|order_id|delivery_company|quantity|   price|ordered_date|             address|
+--------+----------------+--------+--------+------------+--------------------+
|       1| delivery_comp_1|       1| 245. 52|    9-2-2022| Cedar Lane Houst...|
|       2| delivery_comp_2|       2|  114.77|        null|Main Street,New Y...|
|       3| delivery_comp_3|       0|  739.43|   14-3-2022|Main Street,Chica...|
|       4| delivery_comp_0|       1|  878.93|   20/4/2022|                null|
|       5| delivery_comp_1|       2|  481.44|        null|Maple Drive Chica...|
|       6| delivery_comp_2|       0|   78.13|        null|Main Street,Houst...|
|       7| delivery_comp_3|       1|  832.17|   20-2-2022|                null|
|       8| delivery_comp_0|       2|   687.8|    1/4/2022|Maple Drive,Los A...|
|       9| delivery_comp_1|       0|  338.44|   13/4/2022|Cedar Lane Miami,...|
|      10| delivery_comp_2|       1|  46